# 2022-10-27

In [57]:
import requests
from bs4 import BeautifulSoup

from typing import List, Dict, Optional

import pandas as pd

In [3]:
page = requests.get("https://www.lamaistas.lt/visi-receptai")

In [49]:
page.status_code

200

In [8]:
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
print(soup.prettify())

In [25]:
all_ps = soup.find_all("p")

In [26]:
all_ps

[<p>Turinio klausimais rašykite:</p>,
 <p><a href="/cdn-cgi/l/email-protection#bac9d1dbd4cffad6dbd7dbd3c9cedbc994d6ce"><span class="__cf_email__" data-cfemail="9ae9f1fbf4efdaf6fbf7fbf3e9eefbe9b4f6ee">[email protected]</span></a></p>,
 <p>(darbo laikas nuo 8 iki 17 val.)</p>,
 <p>Reklamos klausimais rašykite</p>,
 <p><a href="/cdn-cgi/l/email-protection#daa8bfb1b6bbb7bb9abbb7beb3bdb3aebbb6f4b6ae"><span class="__cf_email__" data-cfemail="126077797e737f7352737f767b757b66737e3c7e66">[email protected]</span></a></p>,
 <p>(darbo laikas nuo 8 iki 17 val.)</p>]

In [16]:
all_divs = soup.find_all("div")

In [28]:
print(type(all_ps[0]))
print(all_ps[0].text)

<class 'bs4.element.Tag'>
Turinio klausimais rašykite:


**labas**

<b>labas</b>

In [48]:
recipes_list_div = soup.find("div", class_="recipesResultSegment")
all_recipes_div = recipes_list_div.find_all("div", class_="frame")

results: List[Dict] = []
for recipe_div in all_recipes_div:
    title = recipe_div.find("div", class_="title").text
    link_to_recipe = recipe_div.find("a")["href"]
    link_to_image = recipe_div.find("img")["src"]

    results.append({
        "title": title.strip(), 
        "link_to_recipe": link_to_recipe,
        "link_to_image": link_to_image
    })

results_df = pd.DataFrame(results)
results_df.to_csv("lamaistas.csv")

In [58]:
def extract_recipes_info(url: str) -> Optional[List[Dict]]:
    resp = requests.get(url)
    if resp.status_code != 200:
        raise Exception("Cannot reach page!")
    soup = BeautifulSoup(resp.content, 'html.parser')
    recipes_list_div = soup.find("div", class_="recipesResultSegment")

    if not recipes_list_div:
        return None    
    all_recipes_div = recipes_list_div.find_all("div", class_="frame")
    results: List[Dict] = []
    for recipe_div in all_recipes_div:
        title = recipe_div.find("div", class_="title").text
        link_to_recipe = recipe_div.find("a")["href"]
        link_to_image = recipe_div.find("img")["src"]

        results.append({
            "title": title.strip(), 
            "link_to_recipe": link_to_recipe,
            "link_to_image": link_to_image
        })

    return results

In [61]:
result: List[Dict] = []

for page in range(2, 30):
    url = f"https://www.lamaistas.lt/visi-receptai/{page}"
    result.extend(extract_recipes_info(url))

pd.DataFrame(result).to_csv("lamaistas.csv")

## Info is puslapio

In [66]:
resp = requests.get("https://www.lamaistas.lt/receptas/naminiu-ledu-sumustiniai-su-braskemis-arba-kitomis-uogomis-81350")
soup = BeautifulSoup(resp.content, "html.parser")

recipe_title = soup.find("div", class_="recipeTitleSegment").find("h1").text
main_recipe_image = soup.find("div", class_="bigImg").find("img").get("src")
about_recipe = soup.find("div", class_="authorAboutRecipeSegment").text

'https://www.lamaistas.lt/uploads/modules/recipes/without-watermark/40168.jpg'

In [79]:
ingredients_div = soup.find("div", class_="ingredients")
ingredients_table = ingredients_div.find("table")
tr_rows = ingredients_table.find_all("tr")
ingredients: List[Dict] = []
for tr_row in tr_rows:
    spans = tr_row.find_all("span")
    ingredients.append({"ingredient": spans[1].text.strip(), "ammount": spans[0].text.strip()})

In [83]:
making_time = soup.find("div", class_="method").find("div", class_="info").text.strip().replace("Paruošimo laikas: ", "")

In [86]:
making_steps: List[Dict] = []
description_divs = soup.find("div", class_="method").find("div", class_="infoA").find_all("div", class_="description")
for step, description_div in enumerate(description_divs):
    making_steps.append({"step": step + 1, "description": description_div.find("div", class_="text").text})


In [90]:
def get_recipe_info(url: str) -> Dict:
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    recipe_title = soup.find("div", class_="recipeTitleSegment").find("h1").text
    main_recipe_image = soup.find("div", class_="bigImg").find("img").get("src")
    about_recipe = soup.find("div", class_="authorAboutRecipeSegment").text

    ingredients_div = soup.find("div", class_="ingredients")
    ingredients_table = ingredients_div.find("table")
    tr_rows = ingredients_table.find_all("tr")
    ingredients: List[Dict] = []
    for tr_row in tr_rows:
        spans = tr_row.find_all("span")
        ingredients.append({"ingredient": spans[1].text.strip(), "ammount": spans[0].text.strip()})

    making_time = soup.find("div", class_="method").find("div", class_="info").text.strip().replace("Paruošimo laikas: ", "")
    making_steps: List[Dict] = []
    description_divs = soup.find("div", class_="method").find("div", class_="infoA").find_all("div", class_="description")
    for step, description_div in enumerate(description_divs):
        making_steps.append({"step": step + 1, "description": description_div.find("div", class_="text").text})

    return {
        "title": recipe_title.strip(),
        "image": main_recipe_image,
        "about": about_recipe.strip(),
        "making_time": making_time,
        "ingredients": ingredients,
        "making_steps": making_steps
    }

In [ ]:
full_recipes 